<a href="https://colab.research.google.com/github/lmmddb/Spotify-Wrapped-/blob/main/main_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#LIBRAIRIES
!pip install spotipy
import os
import json
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.2/354.2 kB 9.0 MB/s eta 0:00:00


In [2]:
# IMPORT DATA
import os
import json
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')

folder_path = '/content/drive/MyDrive/Spotify'

def load_spotify_files(folder_path, pattern="StreamingHistory_music_"):
    dataframes = []

    try:
        files = [f for f in os.listdir(folder_path)
                 if f.startswith(pattern) and f.endswith(".json")]

        for filename in sorted(files):
            file_path = os.path.join(folder_path, filename)
            try:
                with open(file_path, 'r', encoding='utf-8') as file:
                    data = json.load(file)
                    df = pd.json_normalize(data)
                    dataframes.append(df)
                    print(f"✓ {filename} chargé ({len(df)} lignes)")
            except json.JSONDecodeError as e:
                print(f"✗ Erreur dans {filename}: {e}")

        if dataframes:
            combined = pd.concat(dataframes, ignore_index=True)
            print(f"\n📊 Total: {len(combined)} enregistrements")
            return combined
        else:
            print("⚠️ Aucun fichier chargé")
            return pd.DataFrame()

    except FileNotFoundError:
        print(f"❌ Dossier '{folder_path}' introuvable")
        return pd.DataFrame()

# Utilisation
streaming = load_spotify_files(folder_path)
display(streaming.head())

Mounted at /content/drive
✓ StreamingHistory_music_0.json chargé (10000 lignes)
✓ StreamingHistory_music_1.json chargé (10000 lignes)
✓ StreamingHistory_music_2.json chargé (10000 lignes)
✓ StreamingHistory_music_3.json chargé (2075 lignes)

📊 Total: 32075 enregistrements


,endTime,artistName,trackName,msPlayed
0,2024-12-22 16:38,Rema,YAYO,40100
1,2024-12-22 18:21,La Mano 1.9,I SAY,12203
2,2024-12-22 18:22,La Mano 1.9,I SAY,96551
3,2024-12-22 18:25,Kany,Magie,154666
4,2024-12-22 18:27,Rema,Soundgasm,121080


In [3]:
#CLEAN AND PREPA DATA
#Le but est de sélectionner des données pertinentes, à savoir des données concernant l'année 2025 uniquement,
#des musiques que j'ai réellement écouter donc les musiques zappées ne seront pas utilisées
#et essayer de nettoyer au mieux les musiques du type lofi/white noise/rain

#convert to datetime format
streaming["endTime"] = pd.to_datetime(streaming["endTime"])

#filter for only 2025
streaming_2025_df = streaming[streaming["endTime"].dt.year == 2025]

# Définir le seuil (ajustez selon votre analyse)
threshold = 15  # en secondes

# Calculer la durée en secondes si pas déjà fait
streaming['seconds_played'] = streaming_2025_df['msPlayed'] / 1000

# Filtrer : garder seulement les écoutes >= seuil
streaming = streaming_2025_df[streaming['seconds_played'] >= threshold].copy()

print(f"✓ Écoutes de moins de {threshold}s supprimées")
print(f"📊 {len(streaming)} écoutes restantes")
###########################################


✓ Écoutes de moins de 15s supprimées
📊 22127 écoutes restantes


/tmp/ipython-input-1990335975.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  streaming = streaming_2025_df[streaming['seconds_played'] >= threshold].copy()


In [4]:
# Détecter les potentiels sons d'ambiance
keywords_to_check = [
    'rain', 'thunder', 'storm', 'ocean', 'wave', 'water', 'river', 'stream',
    'nature', 'forest', 'bird', 'wind', 'fire', 'crackling',
    'white noise', 'pink noise', 'brown noise', 'ambient noise',
    'asmr', 'binaural', 'meditation', 'relaxation', 'calm', 'peace',
    'yoga', 'zen', 'chakra', 'mindfulness', 'sleep', 'sleeping',
    'healing', 'therapy', 'spa', 'massage',
    'lofi', 'lo-fi', 'lo fi', 'chill beats', 'study beats',
    'frequency', 'hz', 'hertz', 'tone', 'soundscape'
]

def detect_ambient(row):
    text = f"{row.get('trackName', '')} {row.get('artistName', '')}".lower()
    return any(keyword in text for keyword in keywords_to_check)

# Marquer les suspects
streaming['is_suspect'] = streaming.apply(detect_ambient, axis=1)

# Créer la liste des suspects avec leurs stats
suspects = streaming[streaming['is_suspect']].groupby(['trackName', 'artistName']).agg({
    'msPlayed': ['count', 'sum']
}).reset_index()

suspects.columns = ['trackName', 'artistName', 'nb_ecoutes', 'temps_total_ms']
suspects['temps_total_min'] = suspects['temps_total_ms'] / 60000
suspects = suspects.sort_values('nb_ecoutes', ascending=False)

print(f"🔍 {len(suspects)} pistes suspectes détectées\n")
print("="*80)
display(suspects)

# Créer un fichier CSV pour faciliter la sélection
suspects.to_csv('/content/drive/MyDrive/Spotify/suspects_to_review.csv', index=False)
print("\n💾 Liste exportée dans: suspects_to_review.csv")
print("📝 Instructions:")
print("   1. Ouvrez le fichier CSV")
print("   2. Ajoutez une colonne 'supprimer' avec 'oui' ou 'non'")
print("   3. Sauvegardez et réexécutez la cellule suivante")

🔍 400 pistes suspectes détectées



,trackName,artistName,nb_ecoutes,temps_total_ms,temps_total_min
38,Black Noise Thunderstorm,MKG Sound Library,426,64681452,1078.024200
178,Deep Sleep Brown Noise,Bits & Noise,213,34725589,578.759817
66,Brown Noise Dreams,Focus Frequency,167,24349139,405.818983
81,Brown Noise Intelligence,BBVB,64,11985258,199.754300
36,Big Waterfall Brown Noise,Fortuna,41,5891374,98.189567
...,...,...,...,...,...
371,Training Day,DA Uzi,1,159423,2.657050
373,Troubled Waters / Homesick,OG DAYV,1,18576,0.309600
375,Under the Bodhi Tree,Native Zen,1,119693,1.994883
378,Water,Tyla,1,98746,1.645767



💾 Liste exportée dans: suspects_to_review.csv
📝 Instructions:
   1. Ouvrez le fichier CSV
   2. Ajoutez une colonne 'supprimer' avec 'oui' ou 'non'
   3. Sauvegardez et réexécutez la cellule suivante


In [5]:
# Charger vos choix
choix = pd.read_csv('/content/drive/MyDrive/Spotify/suspects_to_review1.csv')

# Vérifier si la colonne existe
if 'supprimer' not in choix.columns:
    print("❌ Ajoutez d'abord une colonne 'supprimer' dans le CSV avec 'oui' ou 'non'")
else:
    # Filtrer ce qui doit être supprimé
    to_remove = choix[choix['supprimer'].str.lower() == 'oui'][['trackName', 'artistName']]

    print(f"🗑️ {len(to_remove)} pistes à supprimer:")
    display(to_remove)

    # Créer un identifiant unique pour matcher
    streaming['track_id'] = streaming['trackName'] + '|||' + streaming['artistName']
    to_remove['track_id'] = to_remove['trackName'] + '|||' + to_remove['artistName']

    # Supprimer
    before = len(streaming)
    streaming = streaming[~streaming['track_id'].isin(to_remove['track_id'])].copy()
    after = len(streaming)

    # Nettoyer
    streaming.drop(['is_suspect', 'track_id'], axis=1, inplace=True)

    print(f"\n✅ Suppression effectuée:")
    print(f"   Avant: {before} écoutes")
    print(f"   Après: {after} écoutes")
    print(f"   Supprimées: {before - after} écoutes")

🗑️ 288 pistes à supprimer:


,trackName,artistName
0,Black Noise Thunderstorm,MKG Sound Library
1,Deep Sleep Brown Noise,Bits & Noise
2,Brown Noise Dreams,Focus Frequency
3,Brown Noise Intelligence,BBVB
4,Big Waterfall Brown Noise,Fortuna
...,...,...
382,Through the Fire,Chaka Khan
388,Brown Noise - 1.400 Hz - Cutoff,Technical Sleep
396,Troubled Waters / Homesick,OG DAYV
397,Under the Bodhi Tree,Native Zen



✅ Suppression effectuée:
   Avant: 22127 écoutes
   Après: 19843 écoutes
   Supprimées: 2284 écoutes
